In [ ]:
from pathlib import Path

data_dir = Path("/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove")

# Utils


In [ ]:
import json
from PIL import Image
import numpy as np


def load_calibration(data_dir: str | Path):
    path = Path(data_dir) / "calibration.json"
    with open(path) as f:
        return json.load(f)


def load_color(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "color" / f"{idx:05}.png"
    return Image.open(path)


def load_pipnet_landmark(data_dir: str | Path, idx: int) -> np.ndarray:
    path = Path(data_dir) / "color/PIPnet_landmarks" / f"{idx:05}.npy"
    return np.load(path)


def load_pipnet_image(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "color/PIPnet_annotated_images" / f"{idx:05}.png"
    return Image.open(path)


def load_mediapipe_landmark(data_dir: str | Path, idx: int) -> np.ndarray:
    path = Path(data_dir) / "color/Mediapipe_landmarks" / f"{idx:05}.npy"
    return np.load(path)


def load_mediapipe_image(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "color/Mediapipe_annotated_images" / f"{idx:05}.png"
    return Image.open(path)


def load_depth_image(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth" / f"{idx:05}.png"
    return Image.open(path)


def load_depth_masked(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_depth.jpg"
    return Image.open(path)


def load_color_masked(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_colormask.jpg"
    return Image.open(path)


def load_normal_masked(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_normal.jpg"
    return Image.open(path)


def load_depth_filter(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_depth_filter.jpg"
    return Image.open(path)


def load_color_filter(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_colormask_filter.jpg"
    return Image.open(path)


def load_normal_filter(data_dir: str | Path, idx: int) -> Image:
    path = Path(data_dir) / "depth_normals_bilateral" / f"{idx:05}_normal_filter.jpg"
    return Image.open(path)

# Masked Images


In [ ]:
image_idx = 0

color = load_color(data_dir=data_dir, idx=image_idx)
display(color)

color_masked = load_color_masked(data_dir=data_dir, idx=image_idx)
display(color_masked)

color_filter = load_color_filter(data_dir, idx=image_idx)
depth_filter = load_depth_filter(data_dir, idx=image_idx)
normal_filter = load_normal_filter(data_dir, idx=image_idx)

display(color_filter)
display(depth_filter)
display(normal_filter)

# color_mask = np.asarray(color_filter) > 0
# depth_mask = np.asarray(depth_filter) > 0
# normal_mask = np.asarray(normal_filter) > 0

# Landmarks


In [ ]:
import numpy as np

image_idx = 100
pipnet_landmarks = load_pipnet_landmark(data_dir, idx=image_idx)
print(f"{pipnet_landmarks.shape=}")
print(pipnet_landmarks[:5, :])
load_pipnet_image(data_dir, idx=image_idx)

In [ ]:
import numpy as np

image_idx = 100
mediapipe_landmarks = load_mediapipe_landmark(data_dir, idx=image_idx)
print(f"{mediapipe_landmarks.shape=}")
print(mediapipe_landmarks[:5, :])
load_mediapipe_image(data_dir, idx=image_idx)

In [ ]:
lms = np.load(
    "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/lms_3d_pip_new/00000.npy"
)
lms.shape

In [ ]:
import open3d as o3d

lms_pc = o3d.io.read_point_cloud(
    "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/lms_3d_pip_new/00000.ply"
)
pc = np.asarray(lms_pc.points)
print(f"{pc.shape=}")
print(pc[:5, :])

# Depth


In [ ]:
import matplotlib.pyplot as plt

depth_image = load_depth_image(data_dir=data_dir, idx=0)
display(depth_image)

# show the depth values
depth_values = np.asarray(depth_image)
plt.hist(depth_values)
plt.show()

In [ ]:
# Note that the depth values are in mm
c = np.asarray(color_masked)
m = c.mean(-1) <= 250
c = c.copy()
c[m] = 0
display(Image.fromarray(c))

plt.hist(depth_values[m])
plt.show()

# Normals and Points in 3D


In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/normals_new/00000.npy"
normal = np.load(path)
normal.shape

In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/points_new/00000.npy"
points = np.load(path)
print(f"{points.shape=}")
print(points[:5, :])

In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/normals_new_maskmouth/00000.npy"
normal = np.load(path)
normal.shape

In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/points_new_maskmouth/00000.npy"
points = np.load(path)
print(f"{points.shape=}")
print(points[:5, :])

# Camera

This is the documentation of the CV2 chair of Kinect intrinsic camera calibration:
https://cvg.cit.tum.de/data/datasets/rgbd-dataset/intrinsic_calibration

This is the information how one can calibrate RGB and depth:
https://github.com/zinsmatt/7-Scenes-Calibration


In [ ]:
path = "/Users/robinborth/Code/GuidedResearch/data/dphm_christoph_mouthmove/camera/c00_color_intrinsic.txt"

intrinsics = np.loadtxt(path)
intrinsics_dict = {
    "color": {
        "cx": intrinsics[0],
        "cy": intrinsics[1],
        "fx": intrinsics[2],
        "fy": intrinsics[3],
        "k1": intrinsics[4],
        "k2": intrinsics[5],
        "k3": intrinsics[6],
        "k4": intrinsics[7],
        "k5": intrinsics[8],
        "k6": intrinsics[9],
        "metric_radius": intrinsics[10],
        "p1": intrinsics[11],
        "p2": intrinsics[12],
    }
}
intrinsics_dict

In [ ]:
intrinsics_dict